<a href="https://colab.research.google.com/github/TewabeTigp/MODIS_LST/blob/main/Drought_index_intensity_frequency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Compute Drought Metrics from Your 20-Year Drought Index**

You need to derive four key drought metrics from your 20-year drought index dataset:

Drought Frequency → Count the number of drought events per grid cell.
Drought Duration → Compute the average length of consecutive drought periods.
Drought Severity → Sum the drought index values for each drought event.
Drought Peak → Record the maximum drought index value in each grid cell.
Assuming your dataset is a time-series of drought indices (e.g., SPI, SPEI) stored as a raster (GeoTIFF) or NetCDF, process the data to extract these metrics.

# **Process Drought Metrics in Python**

Load and Analyze the Drought Index Data
If your drought index is stored as a NetCDF file (.nc) or raster stack (.tif), use xarray and rasterio.

In [ ]:
import numpy as np
import xarray as xr
import rasterio

# Load drought index dataset (replace 'drought_data.nc' with your actual file)
drought_data = xr.open_dataset("drought_data.nc")  # If it's NetCDF
drought_index = drought_data['Drought_Index']  # Select drought index variable

# Convert to NumPy array
drought_values = drought_index.values  # Shape: (Time, Lat, Lon)


# Calculate Drought Frequency
Count the number of drought events where the index is below a threshold (e.g., SPI < -1 or SPEI < -1).

In [ ]:
drought_threshold = -1  # Define drought condition (SPI/SPEI < -1)
drought_events = drought_values < drought_threshold  # Boolean mask
drought_frequency = np.sum(drought_events, axis=0)  # Count occurrences per grid cell


# Calculate Drought Duration
Find the longest continuous drought period at each pixel.

In [ ]:
def max_consecutive_drought(arr, threshold=-1):
    max_duration = 0
    current_duration = 0
    for val in arr:
        if val < threshold:
            current_duration += 1
            max_duration = max(max_duration, current_duration)
        else:
            current_duration = 0
    return max_duration

drought_duration = np.apply_along_axis(max_consecutive_drought, 0, drought_values)


# Calculate Drought Severity
Drought severity is the sum of all negative drought index values for each pixel.

In [ ]:
drought_severity = np.sum(np.where(drought_values < drought_threshold, drought_values, 0), axis=0)


# Calculate Drought Peak
Find the lowest (most extreme negative) value for each pixel.


In [ ]:
drought_peak = np.min(drought_values, axis=0)


# Save Computed Metrics as Raster Files
Convert your computed arrays into GeoTIFFs for mapping.

In [ ]:
def save_raster(output_path, data, reference_raster):
    with rasterio.open(reference_raster) as src:
        meta = src.meta.copy()

    meta.update(dtype=rasterio.float32, count=1)

    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float32), 1)

# Define input reference raster (replace with your raster)
reference_raster = "your_reference.tif"

# Save each metric
save_raster("drought_frequency.tif", drought_frequency, reference_raster)
save_raster("drought_duration.tif", drought_duration, reference_raster)
save_raster("drought_severity.tif", drought_severity, reference_raster)
save_raster("drought_peak.tif", drought_peak, reference_raster)


# Visualize Drought Metrics as Maps
Use matplotlib to plot the rasters.

In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show

def plot_raster(file_path, title, cmap):
    with rasterio.open(file_path) as src:
        data = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

    plt.figure(figsize=(8, 6))
    plt.imshow(data, cmap=cmap, extent=extent)
    plt.colorbar(label=title)
    plt.title(title)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()

# Plot each map
plot_raster("drought_frequency.tif", "Drought Frequency", "OrRd")
plot_raster("drought_duration.tif", "Drought Duration", "YlGnBu")
plot_raster("drought_severity.tif", "Drought Severity", "RdBu_r")
plot_raster("drought_peak.tif", "Drought Peak", "coolwarm")


# Final Map Layout (Multi-Panel Plot)
To create a multi-panel layout like the one in your image:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Define raster files and titles
metrics = ["drought_frequency.tif", "drought_duration.tif", "drought_severity.tif", "drought_peak.tif"]
titles = ["(a) Frequency", "(b) Duration", "(c) Severity", "(d) Peak"]
cmaps = ["OrRd", "YlGnBu", "RdBu_r", "coolwarm"]

for i, ax in enumerate(axes.flat):
    with rasterio.open(metrics[i]) as src:
        data = src.read(1)
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

    im = ax.imshow(data, cmap=cmaps[i], extent=extent)
    plt.colorbar(im, ax=ax)
    ax.set_title(titles[i])
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

plt.tight_layout()
plt.show()


# Export Maps for Publication
Once the maps are generated, you can export them as high-resolution PNG or PDF files.

In [ ]:
plt.savefig("drought_metrics_map.png", dpi=300, bbox_inches='tight')
plt.savefig("drought_metrics_map.pdf", dpi=300, bbox_inches='tight')


# **Convert GeoTIFF Stack to NetCDF**


If you have a folder with 240 monthly drought index rasters, use rasterio and xarray to convert them.

# Import Required Libraries

In [ ]:
import os
import numpy as np
import xarray as xr
import rasterio
from rasterio.plot import show

# Define input folder containing 240 monthly drought index GeoTIFFs
input_folder = "drought_tifs"

# List all raster files (Assume filenames like 'SPI_2001_01.tif', 'SPI_2001_02.tif', ...)
tif_files = sorted([f for f in os.listdir(input_folder) if f.endswith(".tif")])

# Read first raster to get metadata
with rasterio.open(os.path.join(input_folder, tif_files[0])) as src:
    meta = src.meta
    width, height = src.width, src.height
    transform = src.transform
    crs = src.crs
    lon_min, lon_max = src.bounds.left, src.bounds.right
    lat_min, lat_max = src.bounds.bottom, src.bounds.top

# Create latitude and longitude arrays
lon = np.linspace(lon_min, lon_max, width)
lat = np.linspace(lat_min, lat_max, height)


# Stack All Rasters as a Time-Series Array

In [ ]:
# Create empty 3D NumPy array for storing data (Time, Lat, Lon)
drought_array = np.empty((len(tif_files), height, width))

# Read each raster and store in the array
for i, tif_file in enumerate(tif_files):
    with rasterio.open(os.path.join(input_folder, tif_file)) as src:
        drought_array[i, :, :] = src.read(1)  # Read band 1

# Generate time values (Assuming monthly data from Jan 2001 to Dec 2020)
time_values = np.arange(np.datetime64('2001-01'), np.datetime64('2021-01'), np.timedelta64(1, 'M'))

# Create Xarray Dataset
drought_ds = xr.Dataset(
    {
        "Drought_Index": (["time", "lat", "lon"], drought_array)
    },
    coords={
        "time": time_values,
        "lat": lat[::-1],  # Reverse latitude to match the correct orientation
        "lon": lon
    }
)

# Save to NetCDF
drought_ds.to_netcdf("drought_data.nc")

print("NetCDF file 'drought_data.nc' created successfully!")


# Verify NetCDF File

In [ ]:
import xarray as xr

# Load the NetCDF file
drought_ds = xr.open_dataset("drought_data.nc")

# Print dataset structure
print(drought_ds)


# Visualize the NetCDF Data

In [ ]:
import matplotlib.pyplot as plt

# Select a time step (e.g., first month)
drought_ds["Drought_Index"].isel(time=0).plot(cmap="RdBu_r")

plt.title("Drought Index - First Month")
plt.show()
